## Projet 10: Application de recommandation de contenu
### Partie 2: Modèlisation 
Dans ce notebook on vas procéder à effectuer la modélisation des nos données pour mettre en place une systeme de recommandation du contenu.
Nous allons utiliser deux approches principales:
- Content-based filtering
- Collaborative-based filtering

Nous allons nous appuyer sur la libraries [Surpise](https://surpriselib.com/) pour mettre en place nos modèles 

### 1. Import 

#### 1.1 Import des libraries

In [1]:
import os
import pickle
import pandas as pd
import surprise

from sklearn.metrics.pairwise import cosine_similarity
from math import floor
import numpy as np

from surprise import SVD, Dataset, Reader
from surprise.model_selection import GridSearchCV

from surprise import Reader, Dataset, KNNBasic, SVD
from surprise.model_selection import train_test_split
from heapq import nlargest
from surprise import Reader, Dataset, KNNBasic, SVD
from surprise.model_selection import train_test_split
from heapq import nlargest


#### 1.2 Import des données

Maintenant nous allons procéder aves l'importations des libraries que nous allons utiliser pour la modèlisation et puis on vas importer les fichiers des données que nous allons utiliser comme base pour les modèles.

#### 1.2.1 Définition des chemins

In [2]:
data_path = "../data/raw/globocom/"
clicks_path= "../data/raw/globocom/clicks/"

#### 1.2.2 Import fichier avec metadonnées des articles

In [3]:
articles_df = pd.read_csv(data_path + 'articles_metadata.csv')
articles_df.drop(columns=['created_at_ts'], inplace=True)
articles_df.head()

,article_id,category_id,publisher_id,words_count
0,0,0,0,168
1,1,1,0,189
2,2,1,0,250
3,3,1,0,230
4,4,1,0,162


In [4]:
articles_df.columns

Index(['article_id', 'category_id', 'publisher_id', 'words_count'], dtype='object')

#### 1.2.3 Import fichiers avec embeddings des articles

In [5]:
# Ouvrir le fichiers pickle et afficher les 5 premiers lignes
with open(data_path + 'articles_embeddings.pickle', 'rb') as f:
    data = pickle.load(f)

embeddings_df = pd.DataFrame(data)
embeddings_df.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,-0.161183,-0.957233,-0.137944,0.050855,0.830055,0.901365,-0.335148,-0.559561,-0.500603,0.165183,...,0.321248,0.313999,0.636412,0.169179,0.540524,-0.813182,0.286870,-0.231686,0.597416,0.409623
1,-0.523216,-0.974058,0.738608,0.155234,0.626294,0.485297,-0.715657,-0.897996,-0.359747,0.398246,...,-0.487843,0.823124,0.412688,-0.338654,0.320787,0.588643,-0.594137,0.182828,0.397090,-0.834364
2,-0.619619,-0.972960,-0.207360,-0.128861,0.044748,-0.387535,-0.730477,-0.066126,-0.754899,-0.242004,...,0.454756,0.473184,0.377866,-0.863887,-0.383365,0.137721,-0.810877,-0.447580,0.805932,-0.285284
3,-0.740843,-0.975749,0.391698,0.641738,-0.268645,0.191745,-0.825593,-0.710591,-0.040099,-0.110514,...,0.271535,0.036040,0.480029,-0.763173,0.022627,0.565165,-0.910286,-0.537838,0.243541,-0.885329
4,-0.279052,-0.972315,0.685374,0.113056,0.238315,0.271913,-0.568816,0.341194,-0.600554,-0.125644,...,0.238286,0.809268,0.427521,-0.615932,-0.503697,0.614450,-0.917760,-0.424061,0.185484,-0.580292


#### 1.2.4 Import fichier avec les interactions des utilisateurs

In [6]:
def get_all_files_clicks(path):
    clicks_df = pd.DataFrame()
    for file in os.listdir(path):
        df = pd.read_csv(path + file)
        clicks_df = pd.concat([clicks_df, df], axis=0)

    return clicks_df

In [7]:
clicks_df = get_all_files_clicks(clicks_path)

In [8]:
clicks_df['click_timestamp'] = pd.to_datetime(clicks_df['click_timestamp'], unit='ms')
clicks_df['session_start'] = pd.to_datetime(clicks_df['session_start'], unit='ms')
clicks_df.head()

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,93863,1507865792177843,2017-10-13 03:36:32,2,96210,2017-10-13 03:37:12.925,4,3,2,1,21,2
1,93863,1507865792177843,2017-10-13 03:36:32,2,158094,2017-10-13 03:37:42.925,4,3,2,1,21,2
2,294036,1507865795185844,2017-10-13 03:36:35,2,20691,2017-10-13 03:36:59.095,4,3,20,1,9,2
3,294036,1507865795185844,2017-10-13 03:36:35,2,96210,2017-10-13 03:37:29.095,4,3,20,1,9,2
4,77136,1507865796257845,2017-10-13 03:36:36,2,336245,2017-10-13 03:42:13.178,4,3,2,1,25,2


### 2. Content-based Filtering

Le **Content-based Filtering** est une méthode de recommandation qui utilise des informations détaillées sur les éléments pour recommander d'autres éléments similaires. Par exemple, dans un système de recommandation de films, le filtrage basé sur le contenu pourrait utiliser des informations telles que le genre du film, le réalisateur, les acteurs, etc.

**Principe**
L'idée est que si un utilisateur a aimé un certain élément dans le passé, il est probable qu'il aimera à nouveau des éléments similaires à l'avenir. Par conséquent, le système recommande des éléments qui sont similaires aux éléments que l'utilisateur a aimés précédemment.

**Calcul de la similarité**
La similarité entre les éléments est généralement calculée en utilisant des techniques telles que la similarité cosinus ou la distance euclidienne. Les éléments qui sont les plus similaires à ceux que l'utilisateur a aimés sont recommandés.

**Note importante**
Il est important de noter que le filtrage basé sur le contenu ne tient pas compte des opinions d'autres utilisateurs. Il se concentre uniquement sur les préférences de l'utilisateur actuel.

Ici les étapes du code que nous allons utiliser: 

1. Nous allons d'abord identifier les articles que l'utilisateur a déjà lus. Cela se fait en filtrant le DataFrame clicks pour les lignes où le user_id correspond à l'utilisateur donné et en extrayant les valeurs de click_article_id.

2. Si l'utilisateur n'a lu aucun article (c'est-à-dire que la liste des articles lus est vide), nous allons recommander les articles les plus populaires. La popularité est déterminée par le nombre de clics que chaque article a reçu, comme enregistré dans le DataFrame clicks.

3. Si l'utilisateur a lu certains articles, nous allons obtenir les embeddings de ces articles à partir du DataFrame articles.

4. Nous allons ensuite supprimer les articles que l'utilisateur a déjà lus de la liste de tous les articles.

5. Nous allons calculer la similarité cosinus entre les embeddings des articles lus par l'utilisateur et les embeddings de tous les autres articles. Cela donne une matrice de similarité où chaque entrée représente la similarité entre un article lu par l'utilisateur et un autre article.

6. Nous allons ensuite recommander les articles qui sont les plus similaires aux articles lus par l'utilisateur. Nous faisons cela en trouvant la valeur maximale dans la matrice de similarité, qui représente l'article le plus similaire. Les indices de cette valeur maximale sont utilisés pour trouver l'article correspondant dans le DataFrame articles.

7. Nous allons répéter l'étape précédente n fois pour recommander n articles. Après chaque recommandation, nous mettons la similarité de l'article recommandé à 0 dans la matrice de similarité pour nous assurer que le même article n'est pas recommandé à nouveau.

8. Enfin, nous allons retourner une liste des articles recommandés.

In [9]:
def contentBasedRecommendArticle(articles, clicks, user_id, n=5):
    # Get the articles read by the user
    articles_read = clicks[clicks['user_id'] == user_id]['click_article_id'].tolist()
    print(f"Articles read by user {user_id}: {articles_read}")

    # If the user hasn't read any articles, recommend the most popular ones
    if len(articles_read) == 0:
        most_popular_articles = clicks['click_article_id'].value_counts().index.tolist()
        print(f"User {user_id} has not read any articles. Recommending most popular articles: {most_popular_articles[:n]}")
        return most_popular_articles[:n]

    # Get the embeddings of the articles read by the user
    articles_read_embedding = articles.loc[articles_read]
    print(f"Embeddings of articles read by user {user_id}: {articles_read_embedding}")

    # Remove the articles read by the user from the list of articles
    articles = articles.drop(articles_read)
    print(f"Remaining articles after removing articles read by user {user_id}: {articles}")

    # Calculate the cosine similarity between the articles read by the user and the other articles
    matrix = cosine_similarity(articles_read_embedding, articles)
    print(f"Cosine similarity matrix: {matrix}")

    recommendations = []

    # Recommend the articles most similar to the articles read by the user
    for i in range(n):
        coord_x = floor(np.argmax(matrix)/matrix.shape[1])
        coord_y = np.argmax(matrix)%matrix.shape[1]

        recommendations.append(int(articles.index[coord_y]))

        # Set the similarity of the recommended article to 0
        matrix[coord_x][coord_y] = 0

    print(f"Recommendations for user {user_id}: {recommendations}")
    return recommendations

In [10]:
def contentBasedRecommendArticle(articles, clicks, user_id, n=5):
    # Get the articles read by the user
    articles_read = clicks[clicks['user_id'] == int(user_id)]['click_article_id'].tolist()
    print(f"Articles read by user {user_id}: {articles_read}")

    # If the user hasn't read any articles, recommend the most popular ones
    if len(articles_read) == 0:
        most_popular_articles = clicks['click_article_id'].value_counts().index.tolist()
        print(f"User {user_id} has not read any articles. Recommending most popular articles: {most_popular_articles[:n]}")
        return most_popular_articles[:n]

    # Get the embeddings of the articles read by the user
    articles_read_embedding = articles.loc[articles_read]
    print(f"Number of articles read by user {user_id}: {len(articles_read)}")

    # Remove the articles read by the user from the list of articles
    articles = articles.drop(articles_read)
    print(f"Remaining articles after removing articles read by user {user_id}: {len(articles)}")

    # Calculate the cosine similarity between the articles read by the user and the other articles
    matrix = cosine_similarity(articles_read_embedding, articles)

    recommendations = []

    # Recommend the articles most similar to the articles read by the user
    for i in range(n):
        coord_x = floor(np.argmax(matrix)/matrix.shape[1])
        coord_y = np.argmax(matrix)%matrix.shape[1]

        recommendations.append(int(articles.index[coord_y]))

        # Set the similarity of the recommended article to 0
        matrix[coord_x][coord_y] = 0


    return recommendations

In [11]:
user_id = "77136"
n_recommendations = 10

#recommended_articles = contentBasedRecommendArticle(embeddings_df, clicks_df, user_id, n_recommendations)

#print(f"Top {n_recommendations} recommended articles for user {user_id}: {recommended_articles}")

In [12]:
def recommend_articles(articles, clicks, user_id, n=5):
    # Convert user_id and click_article_id to integer type
    clicks['user_id'] = clicks['user_id'].astype(int)
    clicks['click_article_id'] = clicks['click_article_id'].astype(int)
    articles.index = articles.index.astype(int)
    
    # Get the articles read by the user
    articles_read = clicks[clicks['user_id'] == int(user_id)]['click_article_id'].tolist()
    print(f"Articles read by user {user_id}: {articles_read}")

    # If the user hasn't read any articles, recommend the most popular ones
    if len(articles_read) == 0:
        most_popular_articles = clicks['click_article_id'].value_counts().index.tolist()
        print(f"User {user_id} has not read any articles. Recommending most popular articles: {most_popular_articles[:n]}")
        return most_popular_articles[:n]

    # Get the embeddings of the articles read by the user
    articles_read_embedding = articles.loc[articles_read]
    print(f"Number of articles read by user {user_id}: {len(articles_read)}")

    # Remove the articles read by the user from the list of articles
    articles = articles.drop(articles_read)
    print(f"Remaining articles after removing articles read by user {user_id}: {len(articles)}")

    # Calculate the cosine similarity between the articles read by the user and the other articles
    matrix = cosine_similarity(articles_read_embedding, articles)

    recommendations = []

    # Recommend the articles most similar to the articles read by the user
    for i in range(n):
        coord_x = floor(np.argmax(matrix)/matrix.shape[1])
        coord_y = np.argmax(matrix)%matrix.shape[1]

        recommendations.append(int(articles.index[coord_y]))

        # Set the similarity of the recommended article to 0
        matrix[coord_x][coord_y] = 0

    return recommendations

def evaluate_recommendations(articles, clicks, user_id, n=5):
    # Get the user's clicks
    user_clicks = clicks[clicks['user_id'] == int(user_id)]['click_article_id'].tolist()

    # Get the recommendations
    recommendations = recommend_articles(articles, clicks, user_id, n)

    # Calculate precision@k
    relevant_recommendations = [rec for rec in recommendations if rec in user_clicks]
    precision_at_k = len(relevant_recommendations) / n

    return precision_at_k

# Assuming `articles_df` is your articles data and `user_id` is the id of the user you want to evaluate
precision_at_k = evaluate_recommendations(articles_df, clicks_df, user_id, 5)
print(f"Precision@k: {precision_at_k}")

Articles read by user 77136: [336245, 96210, 208150, 283505, 95972, 286108, 123909, 271262, 313920, 31836, 336223, 119193, 207603, 277104, 160974, 300470, 225463, 336221, 271261, 277133]
Number of articles read by user 77136: 20
Remaining articles after removing articles read by user 77136: 364027
Precision@k: 0.0


In [13]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

def recommend_articles(articles, clicks, user_id, n=5):
    # Convert user_id and click_article_id to integer type
    clicks['user_id'] = clicks['user_id'].astype(int)
    clicks['click_article_id'] = clicks['click_article_id'].astype(int)
    articles.index = articles.index.astype(int)
    
    # Get the articles read by the user
    articles_read = clicks[clicks['user_id'] == int(user_id)]['click_article_id'].tolist()
    print(f"Articles read by user {user_id}: {articles_read}")

    # If the user hasn't read any articles, recommend the most popular ones
    if len(articles_read) == 0:
        most_popular_articles = clicks['click_article_id'].value_counts().index.tolist()
        print(f"User {user_id} has not read any articles. Recommending most popular articles: {most_popular_articles[:n]}")
        return most_popular_articles[:n]

    # Get the embeddings of the articles read by the user
    articles_read_embedding = articles.loc[articles_read].values

    # Calculate the cosine similarity between the articles read by the user and all articles
    similarity_matrix = cosine_similarity(articles_read_embedding, articles.values)
    
    print(f"Similarity matrix shape: {similarity_matrix.shape}")

    # Aggregate similarity scores
    similarity_scores = np.mean(similarity_matrix, axis=0)
    
    print(f"Aggregated similarity scores: {similarity_scores}")

    # Get the indices of the top n articles, excluding those already read
    recommendations = []
    sorted_indices = np.argsort(similarity_scores)[::-1]
    for idx in sorted_indices:
        article_id = articles.index[idx]
        if article_id not in articles_read:
            recommendations.append(article_id)
        if len(recommendations) == n:
            break
    
    print(f"Recommended articles: {recommendations}")

    return recommendations

def evaluate_recommendations(articles, clicks, user_id, n=5):
    # Get the user's clicks
    user_clicks = clicks[clicks['user_id'] == int(user_id)]['click_article_id'].tolist()
    print(f"User clicks: {user_clicks}")

    # Get the recommendations
    recommendations = recommend_articles(articles, clicks, user_id, n)
    print(f"Recommendations: {recommendations}")

    # Calculate precision@k
    relevant_recommendations = [rec for rec in recommendations if rec in user_clicks]
    precision_at_k = len(relevant_recommendations) / n

    return precision_at_k

# Example usage
# Assuming `articles_df` is your articles data and `clicks_df` is your clicks data
# and `user_id` is the id of the user you want to evaluate
precision_at_k = evaluate_recommendations(articles_df, clicks_df, user_id, 5)
print(f"Precision@k: {precision_at_k}")




User clicks: [336245, 96210, 208150, 283505, 95972, 286108, 123909, 271262, 313920, 31836, 336223, 119193, 207603, 277104, 160974, 300470, 225463, 336221, 271261, 277133]
Articles read by user 77136: [336245, 96210, 208150, 283505, 95972, 286108, 123909, 271262, 313920, 31836, 336223, 119193, 207603, 277104, 160974, 300470, 225463, 336221, 271261, 277133]
Similarity matrix shape: (20, 364047)
Aggregated similarity scores: [0.00132258 0.00662165 0.00932843 ... 0.99999884 0.99999871 0.99999919]
Recommended articles: [226028, 245736, 245549, 225736, 246041]
Recommendations: [226028, 245736, 245549, 225736, 246041]
Precision@k: 0.0


In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

def build_user_item_matrix(clicks, max_users=1000, max_articles=1000):
    # Limit the number of users and articles for testing
    limited_clicks = clicks[clicks['user_id'].isin(clicks['user_id'].unique()[:max_users]) & clicks['click_article_id'].isin(clicks['click_article_id'].unique()[:max_articles])]
    user_item_matrix = limited_clicks.pivot_table(index='user_id', columns='click_article_id', values='click_timestamp', aggfunc='count').fillna(0)
    return csr_matrix(user_item_matrix.values), user_item_matrix.index, user_item_matrix.columns

def recommend_articles(articles, clicks, user_id, n=5):
    # Convert user_id and click_article_id to integer type
    clicks['user_id'] = clicks['user_id'].astype(int)
    clicks['click_article_id'] = clicks['click_article_id'].astype(int)
    articles.index = articles.index.astype(int)

    # Build user-item interaction matrix
    user_item_matrix, user_index, item_index = build_user_item_matrix(clicks)
    
    if user_id not in user_index:
        # If the user hasn't read any articles, recommend the most popular ones
        most_popular_articles = clicks['click_article_id'].value_counts().index.tolist()
        print(f"User {user_id} has not read any articles. Recommending most popular articles: {most_popular_articles[:n]}")
        return most_popular_articles[:n]

    # Find the index of the user
    user_idx = user_index.get_loc(user_id)
    
    # Compute cosine similarity between users using sparse matrix
    user_similarities = cosine_similarity(user_item_matrix[user_idx], user_item_matrix)
    user_similarities_df = pd.DataFrame(user_similarities, index=user_index, columns=[user_id])
    
    # Get the users most similar to the target user
    similar_users = user_similarities_df[user_id].sort_values(ascending=False).index.tolist()[1:]
    print(f"Users similar to user {user_id}: {similar_users[:5]}")  # Print the top 5 similar users

    # Aggregate the articles read by similar users, excluding those already read by the target user
    articles_read_by_target_user = set(clicks[clicks['user_id'] == user_id]['click_article_id'])
    recommended_articles = clicks[clicks['user_id'].isin(similar_users)]['click_article_id'].value_counts().index.tolist()
    
    # Filter out articles already read by the target user
    recommended_articles = [article for article in recommended_articles if article not in articles_read_by_target_user]
    
    # Return the top n recommendations
    print(f"Recommended articles for user {user_id}: {recommended_articles[:n]}")
    return recommended_articles[:n]

def evaluate_recommendations(articles, clicks, user_id, n=5):
    # Get the user's clicks
    user_clicks = clicks[clicks['user_id'] == int(user_id)]['click_article_id'].tolist()
    print(f"User clicks: {user_clicks}")

    # Get the recommendations
    recommendations = recommend_articles(articles, clicks, user_id, n)
    print(f"Recommendations: {recommendations}")

    # Calculate precision@k
    relevant_recommendations = [rec for rec in recommendations if rec in user_clicks]
    precision_at_k = len(relevant_recommendations) / n

    return precision_at_k

# Example usage with reduced dataset
# Assuming `articles_df` is your articles data and `clicks_df` is your clicks data
# and `user_id` is the id of the user you want to evaluate

# Reduce dataset for testing
reduced_clicks_df = clicks_df.sample(n=10000, random_state=42)  # Adjust this number based on your dataset size
reduced_articles_df = articles_df.loc[articles_df.index.isin(reduced_clicks_df['click_article_id'].unique())]

try:
    precision_at_k = evaluate_recommendations(reduced_articles_df, reduced_clicks_df, user_id, 5)
    print(f"Precision@k: {precision_at_k}")
except Exception as e:
    print(f"An error occurred: {e}")


User clicks: []
User 77136 has not read any articles. Recommending most popular articles: [160974, 272143, 336221, 64329, 234698]
Recommendations: [160974, 272143, 336221, 64329, 234698]
Precision@k: 0.0


### 3. Collaborative-based Filtering

Le **Collaborative-based Filtering** est une méthode de recommandation qui se base sur les comportements passés des utilisateurs pour faire des prédictions sur ce qu'un utilisateur pourrait aimer.

**Principe**
L'idée principale est que si deux utilisateurs ont eu des comportements similaires par le passé (par exemple, ils ont aimé les mêmes films ou acheté les mêmes produits), alors ils sont susceptibles d'avoir des intérêts similaires à l'avenir.

**Types de Filtrage Collaboratif**
Il existe deux types principaux de filtrage collaboratif :

1. **Filtrage Collaboratif Basé sur les Utilisateurs** : Cette méthode trouve des utilisateurs similaires à l'utilisateur cible et recommande des éléments que ces utilisateurs similaires ont aimés.

2. **Filtrage Collaboratif Basé sur les Éléments** : Cette méthode trouve des éléments similaires à ceux que l'utilisateur cible a aimés et recommande ces éléments similaires.

3. **Filtrage Collaboratif Basé sur un Modèle** : Cette méthode utilise des techniques de modélisation, comme la factorisation de matrices ou le clustering, pour prédire l'intérêt d'un utilisateur pour un élément. Elle se base sur les comportements passés de tous les utilisateurs, ainsi que sur les évaluations que l'utilisateur cible a données à d'autres éléments.

**Calcul de la similarité**
La similarité entre les utilisateurs ou les éléments est généralement calculée en utilisant des techniques telles que la corrélation de Pearson ou la similarité cosinus.

**Note importante**
Contrairement au filtrage basé sur le contenu, le filtrage collaboratif ne nécessite pas d'informations détaillées sur les éléments. Il se base uniquement sur les interactions passées entre les utilisateurs et les éléments.

Tout d'abord, nous allons rechercher les meilleurs paramètres pour le modèle en utilisant `GridSearchCV`. `GridSearchCV` est une méthode de recherche exhaustive qui parcourt toutes les combinaisons possibles de paramètres pour trouver celle qui produit le meilleur score de validation croisée.

Dans le contexte de l'apprentissage automatique, les paramètres sont les configurations du modèle que nous ajustons pour améliorer la performance. Par exemple, dans un modèle de forêt aléatoire, les paramètres pourraient inclure le nombre d'arbres dans la forêt (`n_estimators`) et la profondeur maximale des arbres (`max_depth`).

`GridSearchCV` fonctionne en entraînant et en évaluant un modèle pour chaque combinaison de paramètres. Il utilise la validation croisée pour évaluer la performance du modèle, ce qui signifie qu'il divise les données en un ensemble d'entraînement et un ensemble de test, entraîne le modèle sur l'ensemble d'entraînement, puis évalue la performance sur l'ensemble de test.

Une fois que `GridSearchCV` a terminé la recherche, nous pouvons obtenir les meilleurs paramètres en utilisant l'attribut `best_params_`. Nous pouvons ensuite utiliser ces paramètres pour entraîner notre modèle final.

In [15]:
# Create a 'click_count' column
clicks_df['click_count'] = clicks_df.groupby(['user_id', 'click_article_id'])['click_timestamp'].transform('count')

# Load a fraction of the data into a Surprise dataset
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(clicks_df[['user_id', 'click_article_id', 'click_count']].sample(frac=0.1, random_state=42), reader)

# Define the parameter grid
param_grid = {
    'n_factors': [20, 50],
    'n_epochs': [10, 20],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.04]
}

# Run a grid search with cross-validation
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Get the best parameters
best_params = gs.best_params['rmse']

print(f"Best parameters: {best_params}")

Best parameters: {'n_factors': 20, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.04}


In [16]:
# Print the RMSE and MAE of the best model
print(f"Best RMSE: {gs.best_score['rmse']}")
print(f"Best MAE: {gs.best_score['mae']}")

Best RMSE: 0.31750044531091054
Best MAE: 0.03363276957546459


Ici les étapes du code que nous allons utiliser: 

Nous allons d'abord utiliser un sous-ensemble plus petit des données pour le filtrage collaboratif afin d'éviter les problèmes de mémoire. Nous faisons cela en échantillonnant une fraction des click_counts.

Ensuite, nous allons créer un lecteur et un objet de données à partir de ce sous-ensemble de données.

Nous allons diviser les données en ensembles d'entraînement et de test.

Nous allons entraîner un modèle SVD avec les meilleurs paramètres.

Nous allons obtenir la liste des articles lus par l'utilisateur. Nous faisons cela en filtrant les clicks pour les lignes où le user_id correspond à l'utilisateur donné et en extrayant les click_article_id.

Nous allons obtenir la liste de tous les articles à partir de l'index du DataFrame articles.

Nous allons supprimer les articles déjà lus par l'utilisateur de la liste de tous les articles.

Nous allons obtenir les notes prédites pour les articles que l'utilisateur n'a pas encore lus. Nous faisons cela en utilisant le modèle SVD pour prédire la note de chaque article non lu.

Nous allons obtenir les n meilleurs articles. Nous faisons cela en trouvant les n articles avec les notes prédites les plus élevées.

Enfin, nous allons retourner la liste des n meilleurs articles.

In [17]:
def collaborativeFilteringRecommendArticle(articles, clicks, user_id, n=5):
    # Create a new DataFrame that counts the number of times a user clicked on an article
    click_counts = clicks.groupby(['user_id', 'click_article_id']).size().reset_index(name='click_count')

    # Use a smaller subset of data for the collaborative filtering to avoid memory issues
    data_subset = click_counts.sample(frac=0.1, random_state=1)  # adjust the fraction as needed

    # Create a reader and a data object
    reader = Reader(rating_scale=(1, data_subset.click_count.max()))  # assuming a click count of at least 1
    data = Dataset.load_from_df(data_subset, reader)

    # Split the data into train and test sets
    trainset, testset = train_test_split(data, test_size=0.2)


    # Train a SVD model with the best parameters
    algo = SVD(n_factors=best_params['n_factors'], n_epochs=best_params['n_epochs'], lr_all=best_params['lr_all'], reg_all=best_params['reg_all'])
    algo.fit(trainset)

    

    # Get the list of articles read by the user
    articles_read = clicks[clicks['user_id'] == user_id]['click_article_id'].tolist()

    # Get the list of all articles
    all_articles = list(articles.index)

    # Remove the articles already read by the user
    articles_to_predict = [article for article in all_articles if article not in articles_read]

    # Get the predicted ratings for the articles not yet read by the user
    predictions = {article: algo.predict(user_id, article).est for article in articles_to_predict}

    # Get the top n articles
    top_n_articles = nlargest(n, predictions, key=predictions.get)

    return top_n_articles

In [18]:
user_id = '20'
n_recommendations = 5

recommended_articles = collaborativeFilteringRecommendArticle(articles_df, clicks_df, user_id, n_recommendations)

print(f"Top {n_recommendations} recommended articles for user {user_id}: {recommended_articles}")

Top 5 recommended articles for user 20: [225471, 76268, 32730, 298099, 158244]
